# Kecenderungan Pembelajaran Mesin

In [ ]:
# !pip install plotly
# !pip install cufflinks
# !pip install chart-studio

In [ ]:
## import libraries 

from collections import Counter 
import pandas as pd 
import numpy as np 
import string 

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.plotly as py

# import chart_studio.plotly as py

from plotly import tools
import seaborn as sns
init_notebook_mode(connected=True)
from itertools import zip_longest
import string 
import re

from nltk.corpus import stopwords 
from nltk.util import ngrams
import nltk 

In [ ]:
stopwords = stopwords.words('english')

In [ ]:
## dataset preparation
messages = pd.read_csv("../input/ForumMessages.csv")

In [ ]:
messages.head()

In [ ]:
messages.info()

In [ ]:
messages['CreationDate'] = pd.to_datetime(messages['PostDate'])
messages['CreationYear'] = messages['CreationDate'].dt.year
messages['CreationMonth'] = messages['CreationDate'].dt.month
messages['CreationMonth'] = messages['CreationMonth'].apply(lambda x : "0"+str(x) if len(str(x)) < 2 else x)
messages['CreationDay'] = "29"
messages['KernelDate'] = messages["CreationYear"].astype(str) +"-"+ messages["CreationMonth"].astype(str) +"-"+ messages["CreationDay"].astype(str)
messages['Message'] = messages['Message'].fillna(" ")

## function to remove html entities from text
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

## function to clean a text
def clntxt(text):
    text = text.lower()
    text = striphtml(text)
    text = " ".join([c for c in text.split() if c not in stopwords])
    for c in string.punctuation:
        text = text.replace(c, " ")
    text = " ".join([c for c in text.split() if c not in stopwords])
    
    words = []
    ignorewords = ["&nbsp;", "quot", "quote", "www", "http", "com"]
    for wrd in text.split():
        if len(wrd) <= 2: 
            continue
        if wrd in ignorewords:
            continue
        words.append(wrd)
    text = " ".join(words)    
    return text

## function to get top ngrams for a given year
def get_top_ngrams(yr, n, limit):
    # get relevant text
    temp = messages[messages['CreationYear'] == yr]
    text = " ".join(temp['Message']).lower()
    
    # cleaning
    text = striphtml(text)
    text = " ".join([c for c in text.split() if c not in stopwords])
    for c in string.punctuation:
        text = text.replace(c, " ")
    text = " ".join([c for c in text.split() if c not in stopwords])
    
    # ignore 
    words = []
    ignorewords = ["&nbsp;", "quot", "quote", "www", "http", "com"]
    for wrd in text.split():
        if len(wrd) <= 2: 
            continue
        if wrd in ignorewords:
            continue
        words.append(wrd)
    text = " ".join(words)
    
    # tokenize
    token = nltk.word_tokenize(text)
    grams = ngrams(token, n)
    grams = [" ".join(c) for c in grams]
    return dict(Counter(grams).most_common(limit))

def check_presence(txt, wrds):    
    cnt = 0
    txt = " "+txt+" "
    for wrd in wrds.split("|"):
        if " "+wrd+" " in txt:
            cnt += 1 
    return cnt

messages['CMessage'] = messages['Message'].apply(lambda x : clntxt(x))

messages['CreationDay'] = "21"
messages['KernelDate'] = messages["CreationYear"].astype(str) +"-"+ messages["CreationMonth"].astype(str) +"-"+ messages["CreationDay"].astype(str)

## Kecenderungan data sains di Kaggle

Sejumlah kecenderungan telah berubah selama bertahun-tahun di bidang data sains. Kaggle adalah komunitas data sains terbesar dan terpopuler di seluruh dunia. Di paper ini, kita menggunakan meta data Kaggle untuk mengeksplorasi tren data sains selama bertahun-tahun.

## 1. Linear Vs Logistic Regression

Mari kita lihat perbandingan antara regresi linier dan regresi logistik dalam forum diskusi, *kernels* dan jawaban-jawaban di kaggle.

In [ ]:
def plotthem(listed, title):    
    traces = []
    for model in listed:
        temp = messages.groupby('KernelDate').agg({model : "sum"}).reset_index()
        trace = go.Scatter(x = temp["KernelDate"], y = temp[model], name=model.split("|")[0].title(), line=dict(shape="spline", width=2), mode = "lines")
        traces.append(trace)

    layout = go.Layout(
        paper_bgcolor='#fff',
        plot_bgcolor="#fff",
        legend=dict(orientation="h", y=1.1),
        title=title,
        xaxis=dict(
            gridcolor='rgb(255,255,255)',
            range = ['2010-04-21','2021-08-21'],
            showgrid=True,
            showline=False,
            showticklabels=True,
            tickcolor='rgb(127,127,127)',
            ticks='outside',
            zeroline=False
        ),
        yaxis=dict(
            title="Number of Kaggle Discussions",
            gridcolor='rgb(255,255,255)',
            showgrid=False,
            showline=False,
            showticklabels=True,
            tickcolor='rgb(127,127,127)',
            ticks='outside',
            zeroline=False
        ),
    )

    fig = go.Figure(data=traces, layout=layout)
    iplot(fig)
    
## linear vs logistic regression
models = ["linear regression", "logistic regression"]
for col in models:
    messages[col] = messages["CMessage"].apply(lambda x : check_presence(x, col))
plotthem(models, "Kaggle Discussions: Linear vs Logistic")    


In [ ]:
messages.columns

* Dari grafik di atas, kita dapat mengamati bahwa selalu ada lebih banyak diskusi terkait regresi logistik daripada regresi linier. Kecenderungan umum adalah bahwa jumlah diskusi meningkat setiap bulan.
* Salah satu indikasinya adalah ada lebih banyak masalah klasifikasi daripada masalah regresi di Kaggle termasuk kompetisi yang paling populer **Titanic Survival Prediction competition**. Kompetisi ini memiliki jumlah diskusi terbanyak dan merupakan salah satu kompetisi terlama di Kaggle. Ada kompetisi regresi juga: Harga Rumah regresi maju, tetapi orang lebih sering memulainya setelah titanic saja.
* Jumlah diskusi regresi logistik di forum, komentar kernel, dan balasan melonjak ke angka yang tinggi pada Oktober 2017 dan Maret 2018. Salah satu alasannya adalah **Toxic Comments Classification Competition**, di mana sejumlah penulis berbagi informasi yang sangat baik terkait dengan model klasifikasi termasuk regresi logistik.

## 2. The dominance of xgboost

In [ ]:
models = ["decision tree","random forest", "xgboost|xgb", "lightgbm|lgb", "catboost"]
for col in models:
    messages[col] = messages["CMessage"].apply(lambda x : check_presence(x, col))
plotthem(models, "Kaggle Discussions: Tree based models")    


* Sebelum 2014, Model Linier, *decision trees*, dan *Random Forest* sangat populer. Tetapi ketika XgBoost menjadi *open source* pada tahun 2014, ia mendapatkan popularitas dengan cepat dan **mendominasi kompetisi kaggle dan kernel**. Hari ini, xgboost masih digunakan secara mendalam dalam kompetisi dan merupakan bagian dari model pemenang banyak kompetisi. Beberapa contohnya adalah **Otto Group Classification Competition** di mana solusi yang mendapatkan tempat pertama menggunakan xgboost.
* Namun dengan kedatangan **Lightgbm pada tahun 2016**, penggunaan xgboost menurun sampai batas tertentu dan popularitas lightgbm mulai meningkat dengan sangat cepat. Berdasarkan tren peningkatan lightgbm baru-baru ini (ditunjukkan dengan warna merah), kita dapat memperkirakan bahwa itu akan mendominasi beberapa tahun ke depan juga, kecuali jika ada perusahaan lain yang membuka sumber model yang lebih baik. Misalnya, lightgbm digunakan dalam solusi pemenang **Porto Seguro’s Safe Driver Prediction** . Salah satu alasan popularitas gbm ringan adalah implementasi yang lebih cepat dan antarmuka yang sederhana dibandingkan dengan xgboost.
* Misalnya, Catboost baru-baru ini dirilis dan mulai populer.


## 3. Trends of Neural Networks and Deep Learning

In [ ]:
models = ["neural network", "deep learning"]
for col in models:
    messages[col] = messages["CMessage"].apply(lambda x : check_presence(x, col))
plotthem(models, "Kaggle Discussions: Neural Networks vs Deep Learning")    

* Jaringan saraf hadir di industri sejak beberapa dekade tetapi dalam beberapa tahun terakhir tren berubah karena akses ke data dan daya komputasi yang jauh lebih besar.
* Era *deep learning* dimulai pada tahun 2014 dengan kedatangan *library* seperti theano, tensorflow pada tahun 2015, dan keras pada tahun 2016. Jumlah diskusi terkait pembelajaran mendalam meningkat secara teratur dan selalu lebih dari jaringan saraf. Juga, banyak penyedia instans cloud seperti Amazon AWS, Google cloud, dll. menunjukkan kemampuan mereka untuk melatih jaringan saraf *deep* di cloud.
* Model *deeplearning* juga menjadi populer karena banyaknya kompetisi **Image Classification** di Kaggle seperti : Data Science Bowl, kompetisi dari Google dll. Selain itu, model *deeplearning* juga menjadi populer untuk masalah klasifikasi teks misalnya **Quora Duplicate Questions Classification**.
* *Deep learning* juga menjadi populer setiap bulan karena varian model yang berbeda seperti RNN, CNN telah menunjukkan peningkatan besar pada kernel. Juga, pembelajaran transfer dan model pra-terlatih telah menunjukkan hasil yang bagus dalam kompetisi.
* Kaggle dapat meluncurkan lebih banyak kompetisi / taman bermain yang terkait dengan Pemodelan Klasifikasi Gambar karena orang ingin banyak belajar darinya. Tidak lupa bahwa Kaggle telah menambahkan dukungan GPU di kernel yang memfasilitasi penggunaan Deep Learning di kaggle.

## 4. ML Tools used on Kaggle

In [ ]:
models = ["scikit", "tensorflow|tensor flow", "keras", "pytorch"]
for col in models:
    messages[col] = messages["CMessage"].apply(lambda x : check_presence(x, col))
plotthem(models, "Kaggle Discussions: ML Tools")    

* *Scikit Learn* adalah satu-satunya *library* yang digunakan di kaggle untuk tugas pembelajaran mesin, tetapi sejak 2015 tensorflow menjadi populer.
* Di antara alat ML, Keras adalah yang paling populer karena implementasi *deep learning* yang sederhana.

## 5. XgBoost vs Keras

In [ ]:
models = ["xgboost|xgb", "keras"]
for col in models:
    messages[col] = messages["CMessage"].apply(lambda x : check_presence(x, col))
plotthem(models, "Kaggle Discussions: Xgboost vs Deep Learning")    

models = ["cnn|convolution", "lstm|rnn|gru"]
for col in models:
    messages[col] = messages["CMessage"].apply(lambda x : check_presence(x, col))
plotthem(models, "Kaggle Discussions: CNN and LSTM")    

* Di antara kedua teknik populer di Kaggle - xgboost dan deeplearning, xgboost tetap menjadi yang teratas karena lebih cepat dan membutuhkan **less computational infrastructure** daripada jaringan saraf yang sangat kompleks dan lebih dalam.

## 6. What Kagglers are using for Data Visualizations ?

In [ ]:
models = ["matplotlib", "seaborn", "plotly"]
for col in models:
    messages[col] = messages["CMessage"].apply(lambda x : check_presence(x, col))
plotthem(models, "Kaggle Discussions: Python Data Visualization Libraries")    

models = ["ggplot", "highchart", "leaflet"]
for col in models:
    messages[col] = messages["CMessage"].apply(lambda x : check_presence(x, col))
plotthem(models, "Kaggle Discussions: R Data Visualization Libraries")    

* Plotly telah mendapatkan begitu banyak popularitas sejak 2017 dan merupakan salah satu perpustakaan visualisasi data yang paling banyak digunakan di antara kernel. Yang terbaik kedua adalah seaborn yang juga digunakan secara luas. Beberapa kernel visualisasi berkualitas tinggi oleh grandmaster kaggle seperti SRK dan Anistropic dibuat dengan plotly. Secara pribadi, saya juga penggemar berat plotly. :P

## 7. Important Data Science Techniques 

In [ ]:
models = ["exploration|explore|eda" , 'feature engineering', 'parameter tuning|hyperparameter tuning|model tuning|tuning', "ensembling|ensemble"]
for col in models:
    messages[col] = messages["CMessage"].apply(lambda x : check_presence(x, col))
plotthem(models, "Kaggle Discussions: Important Data Science Techniques")    

* Di antara langkah-langkah *data sains* yang penting, pengguna kaggle banyak berfokus pada **Model Ensembling** karena banyak solusi pemenang dalam kompetisi kaggle adalah model ensemble - model campuran dan bertumpuk. Di hampir setiap kernel regresi atau klasifikasi, orang dapat melihat kernel ensemblling. Sebagai contoh - dalam Kompetisi Klasifikasi Komentar toxic, sejumlah besar kernel ensembling dibagikan.
* **Eksplorasi Data** adalah teknik yang penting dan orang-orang mulai menekankan pentingnya eksplorasi di kernel EDA.
* Mengejutkan melihat bahwa diskusi yang berkaitan dengan **Rekayasa Fitur dan Model Tuninig lebih sedikit daripada Ensembling**. Kedua tugas ini memiliki arti paling penting dalam model terbaik dan akurat. Orang cenderung lupa bahwa ensambling hanyalah tahap terakhir dari setiap proses pemodelan tetapi banyak waktu harus diberikan untuk tugas-tugas rekayasa fitur dan penyetelan model.

## 8. Kaggle Components : What people talks about the most 

In [ ]:
models = ["dataset" , 'kernel', 'competition', 'learn']
for col in models:
    messages[col] = messages["CMessage"].apply(lambda x : check_presence(x, col))
plotthem(models, "What is hottest on Kaggle")    

* Komunitas Kaggle telah berbagi sejumlah diskusi terkait kompetisi di forum dan meningkat secara umum.
* Dengan peluncuran kernel pada tahun 2016, penggunaannya meningkat pesat. Pertama kaggler berbagi kernel hanya dalam kompetisi, tetapi dengan lebih fokus pada **kaggle datasets, kernel awards**, jumlah diskusi terkait kernel mulai meningkat dan telah melampaui diskusi terkait kompetisi. Juga, sejumlah **Data Science for Good Challenges** dan **Kernels only competition** telah diluncurkan di kaggle yang merupakan salah satu alasan popularitas kernel.
* Kaggle juga meluncurkan bagian **Kaggle Learn** yang luar biasa yang menjadi populer dan populer tetapi masih tertinggal dari kompetisi, kernel, dan diskusi. Ini karena audiens utamanya adalah pemula dan pemula, tetapi pasti di tahun-tahun mendatang dan dengan lebih banyak penambahan kursus, bagian belajar kaggle akan mencapai level yang sama seperti kompetisi dan kernel.